In [1]:
import h5py
import re
#import hickle as hkl
import torch
import torch.nn as nn
from torch.legacy.nn import Reshape
import graphviz
import torch.nn.functional as F
from torch.autograd import Variable
#from visualize import make_dot
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as utils
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imresize, imread, imshow
import time
import logging
from math import log,sqrt

/home/cvrlab/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
google3=models.inception_v3()

In [ ]:
print(google3)

In [3]:
google3.AuxLogits=nn.Sequential(*list(google3.AuxLogits.children())[:-1])

#google3=nn.Sequential(*list(google3.children())[:-1])

In [ ]:
print(google3.AuxLogits)

In [4]:
def InitializeWeights(mod):
    for m in mod.modules():
        if isinstance(m,nn.Conv2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            #print m.weight.size(), m.out_channels, m.in_channels
            m.weight.data.normal_(0,sqrt(2./n))
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.bias.data.zero_()
    return mod

In [5]:
#__________________lastfc________________
conv1_l = nn.Sequential(nn.BatchNorm2d(2048),nn.ReLU(),nn.Conv2d(2048,1024,(7,7),stride=(1,2)))
conv1_l = InitializeWeights(conv1_l)
conv2_l = nn.Sequential(nn.BatchNorm2d(1024),nn.ReLU(),nn.Conv2d(1024,128,(5,5)))
conv2_l = InitializeWeights(conv2_l)
conv3_l = nn.Sequential(nn.BatchNorm2d(128),nn.ReLU(),nn.Conv2d(128,16,3))
conv3_l = InitializeWeights(conv3_l)
norm1_l = nn.BatchNorm2d(16)
norm1_l = InitializeWeights(norm1_l)
fc1_l= nn.Sequential(nn.ReLU(),nn.Linear(864, 1))




#___________aux_______________
conv1_aux = nn.Sequential(nn.BatchNorm2d(768),nn.ReLU(),nn.Conv2d(768,512,7,stride=(1,2)))
conv1_aux = InitializeWeights(conv1_aux)
conv2_aux = nn.Sequential(nn.BatchNorm2d(512),nn.ReLU(),nn.Conv2d(512,128,11))
conv2_aux = InitializeWeights(conv2_aux)
conv3_aux = nn.Sequential(nn.BatchNorm2d(128),nn.ReLU(),nn.Conv2d(128,64,5))
conv3_aux = InitializeWeights(conv3_aux)
conv4_aux = nn.Sequential(nn.BatchNorm2d(64),nn.ReLU(),nn.Conv2d(64,16,5))
conv4_aux = InitializeWeights(conv4_aux)
conv5_aux = nn.Sequential(nn.BatchNorm2d(16),nn.ReLU(),nn.Conv2d(16,8,3))
conv5_aux = InitializeWeights(conv5_aux)
norm1_aux = nn.BatchNorm2d(8)
norm1_aux = InitializeWeights(norm1_aux)
fc1_aux= nn.Sequential(nn.ReLU(),nn.Linear(1344, 1))

In [6]:
class MyModel4(nn.Module):
    def __init__(self, pretrained_model):
        super(MyModel4, self).__init__()
        
        self.last_fc = nn.Sequential(*list(pretrained_model.children())[:-1])
        self.aux = nn.Sequential(*list(pretrained_model.children())[:-4])
        ########
        self.conv1_l = conv1_l
        self.conv2_l = conv2_l
        self.conv3_l = conv3_l
        self.norm1_l = norm1_l
        self.fc1_l = fc1_l  
        #########
        self.conv1_aux = conv1_aux
        self.conv2_aux = conv2_aux
        self.conv3_aux = conv3_aux
        self.conv4_aux = conv4_aux
        self.conv5_aux = conv5_aux
        self.norm1_aux = norm1_aux
        self.fc1_aux = fc1_aux
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    
    def forward(self, x):
        #______________InceptionE_____________
        y=x
        z=x
        y = self.last_fc(x)
        y = self.conv1_l(y)
        y = self.conv2_l(y)
        y = self.conv3_l(y)
        y = self.norm1_l(y)
        y = y.view(-1, self.num_flat_features(y))
        y = self.fc1_l(y)
        #_____________________AuxLogits___________
        z = self.aux(z)
        z = self.conv1_aux(z)
        z = self.conv2_aux(z)
        z = self.conv3_aux(z)
        z = self.conv4_aux(z)
        z = self.conv5_aux(z)
        z = self.norm1_aux(z)
        z = z.view(-1, self.num_flat_features(z))
        z = self.fc1_aux(z)
        #print(y)
        #print(z)
        return y,z

In [7]:
net= MyModel4(google3)
#print(aux_o)
#print(incep_e)
#print(type(b))
#print(b)

In [ ]:
print(google3)

In [ ]:
input=Variable(torch.randn(1,3,180,320))
tic=time.time()
incep_eo,aux_o=net(input)
toe=time.time()
print(toe-tic)
print(incep_eo.size())
print(aux_o.size())

In [ ]:
print(incep_eo)
print(aux_o)

In [15]:
sum1 = 0
        
print("Number of layers ---> ",len(list(google3.parameters())))
for params in google3.parameters():
    if params.requires_grad == True:
        sum1 += params.numel()
    
print("Total number of parameters ---> ",sum1)

Number of layers --->  290
Total number of parameters --->  26392264


In [16]:
sum1 = 0
        
print("Number of layers ---> ",len(list(net.parameters())))
for params in net.parameters():
    if params.requires_grad == True:
        sum1 += params.numel()
    
print("Total number of parameters ---> ",sum1)

Number of layers --->  328
Total number of parameters --->  157841466


In [ ]:
#############      rough
ind1 = np.random.randint(0,24)
print(ind1)

In [8]:
file = h5py.File('./DATASET/CODE/NewTrainData_59_cor__35000.h5')
xtrainT = torch.from_numpy(np.array(file['xtrain'],dtype=np.float32)).float()
ytrainT = torch.from_numpy(np.array(file['ytrain'],dtype=np.float32)).float()
#xtrain = np.array(file['xtrain'],dtype=np.float32)
#ytrain = np.array(file['ytrain'],dtype=np.float32)

In [ ]:
file = h5py.File('./DATASET/CODE/NewTestData_22_cor_random_2.h5')
xtestT = torch.from_numpy(np.array(file['xtest'],dtype=np.float32)).float()
ytestT = torch.from_numpy(np.array(file['ytest'],dtype=np.float32)).float()
#xtest = np.array(file['xtest'],dtype=np.float32)
#ytest = np.array(file['ytest'],dtype=np.float32)

In [9]:
def batch_rgb_to_bgr(batch):
    #print(batch.size())
    (r, g, b) = torch.chunk(batch, 3, 1)
    #print(r.size())
    batch1 = torch.cat((b, g, r),1)
    #print(batch1.size())
    return batch1

In [ ]:
#xtrainT = batch_rgb_to_bgr(xtrainT)
#xtestT = batch_rgb_to_bgr(xtestT)
#print(xtrainT.size(), xtestT.size())
print(xtrainT.size())

In [ ]:
#xtrainT = torch.div(xtrainT,255.0)
#xtestT = torch.div(xtestT,255.0)

In [ ]:
# denom = [255.0,255.0,255.0]
# for t, m in zip(xtrainT, denom):
#          t.div_(m)

In [ ]:
print(xtrainT.size(), ytrainT.size(), xtestT.size(), ytestT.size())

In [10]:
class Normalize(object):
    """
    Normalize an tensor image with mean and standard deviation.
    Given mean: (R, G, B) and std: (R, G, B),
    will normalize each channel of the torch.*Tensor, i.e.
    channel = (channel - mean) / std
    Args:
        mean (sequence): Sequence of means for R, G, B channels respecitvely.
        std (sequence): Sequence of standard deviations for R, G, B channels
            respecitvely.
    """

    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        # TODO: make efficient
        for t, m, s in zip(tensor, self.mean, self.std):
            t.sub_(m).div_(s)
        return tensor

In [11]:
mn = [0.406,0.456,0.485]
sd = [0.225,0.224,0.229]
norm = Normalize(mn,sd)
#xtrainT = norm(xtrainT)
#xtestT = norm(xtestT)

In [ ]:
#print(torch.min(xtrainT), torch.max(xtrainT), torch.min(xtestT), torch.max(xtestT))
print(torch.min(xtrainT), torch.max(xtrainT))

In [ ]:
print(xtrainT[1,0,0,0])
type(xtrainT)
#xtrainT.index(255.0)

In [12]:
def train(model, loss, optimizer, x_val, y_val,batch_size):
    x = Variable(x_val,requires_grad = False).cuda()
    y = Variable(y_val,requires_grad = False).cuda()
    
    optimizer.zero_grad()
    x = batch_rgb_to_bgr(x)
    x = torch.div(x,255.0)
    mn = [0.406,0.456,0.485]
    sd = [0.225,0.224,0.229]
    x[:,0,:,:] = (x[:,0,:,:]-mn[0])/sd[0]
    x[:,1,:,:] = (x[:,1,:,:]-mn[0])/sd[0]
    x[:,2,:,:] = (x[:,2,:,:]-mn[0])/sd[0]
    #x=torch.div(torch.sub(x-mn),sd)
    
    fx1,fx2 = model.forward(x)
    
    #print fx.data[0][0][64][87]
    #fx = model5.forward(Variable(xtest2[start:end], volatile=True).cuda())
    ##output = loss.forward(fx,y,validPixel,batch_sz)
    output = loss.forward(fx1,fx2,y)
    #output = loss(fx, y)
    output.backward()
    
    optimizer.step()
    
    return output.data[0]

In [13]:

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        
    def forward(self,inp1,inp2, tar):
        #target is the ground truth value...
        #k = torch.mean(inp[:,0])
        '''
        if (k >= 1.48 and k <= 1.65):
            diff = torch.abs(tar[:,1]-inp[:,1])
            loss = torch.mean(torch.pow(diff,2))
        else:
        '''
        diff1 = torch.abs(tar[:,0]-inp1[:,0]) #*(180/np.pi)
        diff2 = torch.abs(tar[:,0]-inp2[:,0])
        diff = diff1+diff2
        loss = torch.mean(diff)
        #print(loss)
        return loss
        '''
        c1 = c.data[0] 
        temp = diff > c1
        check1 = torch.prod(temp)
        
        if check1 == 0:
            lossval = torch.mean(diff)
        else:
            temp4 = torch.pow(diff,2)
            d = torch.pow(c,2)
            temp4 = temp4.add(d.expand_as(temp4))
            lossval = torch.mean(temp4/(2*c))
        return lossval
        '''

In [14]:
#MUST UNCOMMENT BELOW LINE...
    
net = net.cuda()

#loading the model after the weights of epoch50.. to check what loss the model gives if lr is taken as 0.0001
optimizer = optim.SGD(net.parameters(), lr=0.0004, momentum=0.9)

#criterion = RMSELoss()
#criterion = BerhuLoss()
#criterion = EuclideanLoss()
#criterion = nn.MSELoss()
#criterion = CosineLoss()
#criterion = torch.nn.MSELoss(size_average=False)
criterion = CustomLoss()
#criterion = BerhuLoss()
#criterion = CosineLoss()
criterion.cuda()

currepochloss =0.01148934#float('Inf')
#epochs, n_examples, i, batch_size, flag = 1,5900, 0, 5, 0
epochs, n_examples, i, batch_size, flag = 1, 35000, 0, 5, 0


while i != epochs:
    since = time.time()
    cost, batchloss = 0.0, 0.0
    num_batches = n_examples//batch_size
    #print num_batches    #indices = np.random.permutation(5600)
    #indices = np.random.permutation(3524)
    
    #indices = np.random.permutation(5900)
    indices = np.random.permutation(n_examples)
    samplesUnprocessed = np.size(indices)
    
    #batchwise training starts here...
    for k in range(num_batches):
        since1 = time.time()
       # print("bacth number:"+str(k))
        xtrain3 = torch.FloatTensor(batch_size,3,180,320)
        ytrain3 = torch.FloatTensor(batch_size,1)
        ##validPixel = torch.FloatTensor(batch_size,480,640)
        
        for ind in range(batch_size):
            #ind1 = np.random.randint(0,5599)
            ind1 = np.random.randint(0,samplesUnprocessed)
            #ind1 = np.random.randint(0,794)
            #ind1 = np.random.randint(0,794)            
            newxind = indices[ind1]            
            xtrain3[ind] = xtrainT[newxind]
            ytrain3[ind] = ytrainT[newxind,0,0]
            ##validPixel[ind] = imgValidTrain2[newxind]
            
            #print ytrain3[ind,0,0,0], ytrain2[newxind,0,0,0]
            indices = np.delete(indices,ind1)
            samplesUnprocessed = samplesUnprocessed - 1
        
        #start, end = k*batch_size, (k+1)*batch_size
        #batchloss = train(model5,criterion, optimizer, xtrain3, ytrain3, validPixel,batch_size)
        batchloss = train(net,criterion, optimizer, xtrain3, ytrain3, batch_size)
        batch_time = time.time() - since1
        #cost += batchloss
        cost = (cost*k+batchloss)/(k+1)
        #print k,cost
        #print("No. of samples UnProcessed "+str(samplesUnprocessed))
        
    time_elapsed = time.time() - since
    epochloss = cost #/num_batches
    
    if epochloss < currepochloss:
        print('save the weights')
        torch.save(net.state_dict(),"./weights/CustomLoss_new/CustomLoss_new_35000_INCEP_NET_new_ANGLE_1_epochs.pth")
        flag = 0
        currepochloss = epochloss
    else:
        flag += 1
        
        if flag == 5:
            for p in optimizer.param_groups:
                lr2 = p['lr']
            newlr = lr2/5
            
            if newlr < 1e-15:
                print("Cant decrease further!!")
                newlr = 1e-15
            flag = 0 
            optimizer = optim.SGD(net.parameters(), lr=newlr, momentum=0.9)
            print("Learning rate changed from "+str(lr2)+" to "+str(newlr))
            
        print("Loss "+str(epochloss)+" is bigger than Loss "+str(currepochloss)+" in the prev epoch ")
        
    print('Loss = {:.8f} at epoch {:d} completed in {:.0f}m {:.0f}s'.format(epochloss,(i+1),(time_elapsed//60),(time_elapsed%60)))
    i += 1 

Loss 0.986270849640883 is bigger than Loss 0.01148934 in the prev epoch 
Loss = 0.98627085 at epoch 1 completed in 107m 11s


In [ ]:

for params in optimizer.param_groups:
    print(params['lr'])

In [ ]:
net = net.cuda()
net.load_state_dict(torch.load("./weights/CustomLoss_new/ANGLE/CustomLoss_new_35000_107_epochs.pth"))

In [ ]:
#testing of the architecture...
num_batches = 0
#6 evenly divides the test batch size..
test_batch_size = 20
n_examples = 600
#finalpred = Variable(torch.zeros((n_examples,3,120,160)))
finalpred = Variable(torch.zeros((n_examples,1)))
print("finalpred size is ---> ", finalpred.size())

num_batches = n_examples//test_batch_size
print("num of batches --->", num_batches)
for k in range(num_batches):
    start, end = k*test_batch_size, (k+1)*test_batch_size
    output = net.forward(Variable(xtestT[start:end], volatile=True).cuda())
    finalpred[start:end] = output

In [ ]:
data1 = finalpred.data.numpy()
print(data1.shape)

In [ ]:
dif = torch.abs(finalpred.data[:,0]-ytestT[:,0,0])
dif1 = torch.abs((finalpred.data[:,0]-ytestT[:,0,0])/ytestT[:,0,0])
print(dif.size())
#np.savetxt("diff.csv", dif.numpy(), delimiter=",")

MSElossRad = torch.mean(torch.pow(dif,2))
ABSlossRad = torch.mean(dif)
RELlossRad = torch.mean(dif1)
MSElossDeg = MSElossRad*(180/np.pi)
ABSlossDeg = ABSlossRad*(180/np.pi)
RELlossDeg = RELlossRad*(180/np.pi)
print("MSElossRad=="+str(MSElossRad),"ABSlossRad=="+str(ABSlossRad),"RELlossRad"+str(RELlossRad))
print("MSElossDeg=="+str(MSElossDeg),"ABSlossDeg=="+str(ABSlossDeg),"RELlossDeg"+str(RELlossDeg))